In [1]:
import pandas as pd
import os
import glob
import re
import json

## Utility functions

In [2]:
def write_json(data_dict,filename,data_write_path):
    '''
    Write JSON file into designated raw data folder.
    '''
    
    # Serializing json
    json_object = json.dumps(data_dict, indent=4)

    # Writing to .json
    with open(data_write_path + filename, "w") as outfile:
        outfile.write(json_object)

## Make nested JSON from all the yearly results data

In [1]:
combined_df = pd.read_csv('./data/cleaned/combined/combined_results_parliament.csv', converters = {'parliament_code_digits': str})
combined_df['results_added'] = 1
# ge15_candidates_clean_df = pd.read_csv('./data/cleaned/combined/combined_results_parliament_ge15.csv', converters = {'parliament_code_digits': str})
# ge15_incumbents_clean_df = pd.read_csv('./data/cleaned/malaysiakini_newslab/ge15_incumbents_clean.csv', converters = {'parliament_code_digits': str})
# ge15_incumbents_clean_df['results_added'] = 1

# combined_df = pd.concat([combined_df, ge15_incumbents_clean_df, ge15_candidates_clean_df])

combined_df = pd.read_csv('./data/raw/Thevesh/prn15.csv', converters = {'parliament_code_digits': str, 'dunCode': str, 'threeDunCode': str})

combined_df.fillna('',inplace=True)
# combined_df.sort_values(by=['year','parliament_code_digits','votes'],ascending=[True,True,False],inplace=True)
combined_df.sort_values(by=['year','dunCode','votes'],ascending=[True,True,False],inplace=True)

NameError: name 'pd' is not defined

In [4]:
combined_df

,year,name,coalition,party_code,votes,vote_share,parliament_code_digits,constituency,state,winner,total_votes,results_added,Unnamed: 0,gender,spr_id
0,2004,DATUK SERI AZMI KHALID,BN,UMNO,18322,66.917458,001,PADANG BESAR,Perlis,1.0,27380,1,,,
1,2004,WAN KHARIZAL WAN KHAZIM,BA,PAS,9058,33.082542,001,PADANG BESAR,Perlis,0.0,27380,1,,,
2,2004,DATUK RADZI SHEIKH AHMAD,BN,UMNO,22498,69.335552,002,KANGAR,Perlis,1.0,32448,1,,,
3,2004,ISHAR SAAD,BA,PAS,9950,30.664448,002,KANGAR,Perlis,0.0,32448,1,,,
4,2004,DATUK SERI SYED RAZLAN SYED PUTRA JAMALULLAIL,BN,UMNO,17367,55.149090,003,ARAU,Perlis,1.0,31491,1,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,2022 Incumbents,LUKANISMAN AWANG SAUNI,GPS,PBB,0,0.000000,218,SIBUTI,Sarawak,1.0,0,1,210,,
217,2022 Incumbents,MICHAEL TEO YU KENG,PH,PKR,0,0.000000,219,MIRI,Sarawak,1.0,0,1,217,,
209,2022 Incumbents,ANYI NGAU,GPS,PDP,0,0.000000,220,BARAM,Sarawak,1.0,0,1,209,,
198,2022 Incumbents,HASBI HABIBOLLAH,GPS,PBB,0,0.000000,221,LIMBANG,Sarawak,1.0,0,1,198,,


In [5]:
combined_df.columns

Index(['year', 'name', 'coalition', 'party_code', 'votes', 'vote_share',
       'parliament_code_digits', 'constituency', 'state', 'winner',
       'total_votes', 'results_added', 'Unnamed: 0', 'gender', 'spr_id'],
      dtype='object')

In [6]:
combined_df['coalition'].value_counts().index

Index(['BN', '', 'PH', 'PR', 'PN', 'GS', 'BA', 'GTA', 'GPS', 'USA', 'GRS'], dtype='object')

In [7]:
parties = combined_df['party_code'].value_counts().index
parties = parties.to_list()
parties.sort()
parties

['AMANAH',
 'ANAK NEGERI',
 'BERJASA',
 'BERSAMA',
 'BERSATU',
 'BERSEKUTU',
 'BN',
 'DAP',
 'DIRECT',
 'GERAKAN',
 'IKATAN',
 'IMAN',
 'IND',
 'INDEPENDENT',
 'IPF',
 'KDM',
 'KIMMA',
 'KITA',
 'LDP',
 'MCA',
 'MDP',
 'MIC',
 'MMSP',
 'MUDA',
 'MUP',
 'MUPP',
 'MYPPP',
 'PAP',
 'PAS',
 'PASOK',
 'PBB',
 'PBDS',
 'PBDSB',
 'PBK',
 'PBM',
 'PBRS',
 'PBS',
 'PCM',
 'PCS',
 'PDP',
 'PEACE',
 'PEJUANG',
 'PERPADUAN',
 'PFP',
 'PHRS',
 'PKAN',
 'PKR',
 'PPBM',
 'PPP',
 'PPRS',
 'PRM',
 'PRS',
 'PSB',
 'PSM',
 'PUR',
 'PUTRA',
 "S'WAK STAR",
 'SABAH STAR',
 'SAPP',
 'SEDAR',
 'SETIA',
 'SNAP',
 'SPDP',
 'STAR',
 'SUPP',
 'SWP',
 'UMNO',
 'UPKO',
 'VACANT',
 'WARISAN']

In [8]:
combined_df_year_district = combined_df.set_index(['year','parliament_code_digits'])

In [9]:
year_district_pairs = combined_df_year_district.index.unique().to_list()

In [10]:
nested_year_district = {}

In [11]:
for year_district in year_district_pairs:

  [year,district] = year_district
  
  if year not in nested_year_district:
    nested_year_district[year] = {}
  
  nested_year_district[year][district] = {}

  district_columns = ['total_votes','constituency','state','results_added']

  result_columns = ['name','party_code','coalition','votes','vote_share','winner','gender']

  nested_year_district[year][district]['info'] = combined_df_year_district.loc[(year,district),:][district_columns].to_dict('records')[0]

  nested_year_district[year][district]['results'] = combined_df_year_district.loc[(year,district),:][result_columns].to_dict('records')

  winner_list = [row for row in nested_year_district[year][district]['results'] if row['winner'] == 1]
  winner = ''
  if len(winner_list) > 0:
    winner = winner_list[0]
  nested_year_district[year][district]['winner'] = winner

    

In [12]:
write_json(nested_year_district,'nested_results_parliament.json','./data/cleaned/combined/')

## Make Parties data into nested JSON

In [13]:
parties_df = pd.read_csv('./data/cleaned/combined/party_info.csv')
parties_df.fillna('',inplace=True)

In [14]:
parties_df

,year,coalition,coalition_name,coalition_name_bahasa,party,party_name,party_color,coalition_color
0,2004,BA,Alternative Front,Barisan Alternatif,PAS,Pan-Malaysian Islamic Party,#009000,#ed161e
1,2004,BA,Alternative Front,Barisan Alternatif,PKR,People's Justice Party,#00ADEF,#ed161e
2,2004,BN,National Front,Barisan Nasional,UMNO,United Malays National Organisation,#E62020,#000080
3,2004,BN,National Front,Barisan Nasional,MCA,Malaysian Chinese Association,#0000FF,#000080
4,2004,BN,National Front,Barisan Nasional,GERAKAN,Malaysian People's Movement Party,#FF0000,#000080
...,...,...,...,...,...,...,...,...
163,2022 Incumbents,PH,Alliance of Hope,Pakatan Harapan,MUDA,Malaysian United Democratic Alliance,#000000,#ED1C24
164,2022 Incumbents,PH,Alliance of Hope,Pakatan Harapan,UPKO,United Pasokmomogun Kadazandusun Murut Organis...,#ABCDFF,#ED1C24
165,2022 Incumbents,BN,National Front,Barisan Nasional,PBRS,United Sabah People's Party,#FEDF00,#000080
166,2022 Incumbents,GRS,Sabah People's Coalition,Gabungan Rakyat Sabah,PBS,United Sabah Party,#ADD8E6,#91cbf2


In [15]:
parties_df_year_party = parties_df.set_index(['year','party'])

In [16]:
year_party_pairs = parties_df_year_party.index.unique().to_list()

In [17]:
parties_nested_year_party = {}

In [18]:
for year_party in year_party_pairs:

  [year,party] = year_party
  
  if year not in parties_nested_year_party:
    parties_nested_year_party[year] = {}
  
  parties_nested_year_party[year][party] = parties_df_year_party.loc[(year,party),:].to_dict('records')[0]



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:1761: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [19]:
write_json(parties_nested_year_party,'parties_nested.json','./data/cleaned/combined/')